In [12]:
import ast
import itertools
from tqdm import tqdm
import numpy as np
import pandas as pd
import seaborn as sns
import rdkit.Chem.AllChem as Chem
from rdkit.Chem import AllChem
import matplotlib.pyplot as plt

In [2]:
# cleanup
df_info = pd.read_csv('data/hMOF_CO2_info.csv')
df_info = df_info.dropna() # drop entries containing 'NaN'
df_info = df_info[df_info.CO2_wc_001>0] # only keep entries with positive CO2 working capacity
df_info = df_info[~df_info.MOFid.str.contains('ERROR')] # drop entries with error
df_info = df_info[~df_info.MOFid.str.contains('NA')] # drop entries with NA

In [6]:
# get node and linker information
metal_eles = ['Zn', 'Cu', 'Mn', 'Zr', 'Co', 'Ni', 'Fe', 'Cd', 'Pb', 'Al', 'Mg', 'V',
       'Tb', 'Eu', 'Sm', 'Tm', 'Gd', 'Nd', 'Dy', 'La', 'Ba', 'Ga', 'In',
       'Ti', 'Be', 'Ce', 'Li', 'Pd', 'Na', 'Er', 'Ho', 'Yb', 'Ag', 'Pr',
       'Cs', 'Mo', 'Lu', 'Ca', 'Pt', 'Ge', 'Sc', 'Hf', 'Cr', 'Bi', 'Rh',
       'Sn', 'Ir', 'Nb', 'Ru', 'Th', 'As', 'Sr']

# get a list of metal nodes & create a new column named "metal_nodes"
metal_nodes = []
organic_linkers = []
for i,mofid in tqdm(enumerate(df_info.MOFid)):
    sbus = mofid.split()[0].split('.')
    metal_nodes.append([c for c in sbus if any(e in c for e in metal_eles)][0])
    organic_linkers.append([c for c in sbus if any(e in c for e in metal_eles)==False])

df_info['metal_node'] = metal_nodes
df_info['organic_linker'] = organic_linkers

107449it [00:02, 39452.56it/s]


In [7]:
unique_nodes = [n for n in list(df_info['metal_node'].unique()) if len(n)<=30] # node smiles should be shorter then 30 strings
df_info = df_info[df_info['metal_node'].isin(unique_nodes)] # filter df_info based on unique_nodes
freq = [df_info['metal_node'].value_counts()[value] for value in list(df_info.metal_node.unique())] # get frequency of unique nodes
df_freq = pd.DataFrame({'node':list(df_info.metal_node.unique()),'freq':freq})
print('most frequently occuring node:')
print(df_freq)
unique_node_select = ['[Zn][Zn]', '[Cu][Cu]', '[Zn][O]([Zn])([Zn])[Zn]', '[V]'] # manually select the most occuring nodes
df_info_select = df_info[df_info['metal_node'].isin(unique_node_select)] # select df_info with node only in list(unique_node_select)
# output df for each node to a separate csv files
for n in unique_node_select:
    df_info_select_node = df_info[df_info.metal_node == n]
    df_info_select_node.to_csv(f'data/data_by_node/{n}.csv',index=False)

most frequently occuring node:
                        node   freq
0                   [Zn][Zn]  28529
1                   [Cu][Cu]  29714
2    [Zn][O]([Zn])([Zn])[Zn]  43874
3                        [V]   5197
4                 [O][V]O[V]     46
5                  [V]1[V]O1     50
6                    [V]O[V]     32
7            [V]O[V]O[V]O[V]      1
8                       [Zn]      2
9          [V]1O[V][V]O[V]O1      1
10  F[Zn][O]([Zn])([Zn])[Zn]      1
11                 [Cu][CuH]      1
12         [Zr][O]([Zr])[Zr]      1


In [13]:
# load data
for node in unique_node_select:
    input_data_path = f'data/data_by_node/{node}.csv' 
    output_data_path = f'data/data_high_wc/{node}.csv'

    df = pd.read_csv(input_data_path)

    # select entries with high working capactiy
    df_high_wc = df[df['CO2_wc_01'] >=2]

    # select entries with three parsed linker
    len_linkers = [len(ast.literal_eval(df_high_wc['organic_linker'].iloc[i])) for i in range(len(df_high_wc['organic_linkers']))]
    df_high_wc['len_linkers'] = len_linkers
    df_high_wc_select = df_high_wc[df_high_wc.len_linkers==3]
    df_high_wc_select.to_csv(output_data_path,index=False)

    # get list of smiles strings for all linkers

    list_smiles = [ast.literal_eval(i) for i in df_high_wc_select['organic_linkers']]
    all_smiles = list(itertools.chain(*list_smiles))
    print(len(all_smiles))

    # output to sdf
    conformer_sdf_path = f'data/conformers_{node}.sdf'

    writer = Chem.SDWriter(conformer_sdf_path)
    for smile in tqdm(all_smiles):
        try:
            mol = Chem.AddHs(Chem.MolFromSmiles(smile))
            conformers = AllChem.EmbedMultipleConfs(mol, numConfs=1)
            conformer = mol.GetConformer(0)
            for cid in range(mol.GetNumConformers()):
                writer.write(mol, confId=cid)
        except:
            pass

/var/folders/k3/l23j0sk144l_frxntwwtfpg40000gn/T/ipykernel_40325/3834039767.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_high_wc['len_linkers'] = len_linkers


2502


  1%|          | 27/2502 [00:00<01:00, 40.88it/s] [22:02:35] Explicit valence for atom # 1 N, 4, is greater than permitted
[22:02:35] Explicit valence for atom # 15 N, 4, is greater than permitted
 13%|█▎        | 328/2502 [00:03<00:19, 112.96it/s][22:02:38] Explicit valence for atom # 2 N, 4, is greater than permitted
[22:02:38] Explicit valence for atom # 3 N, 4, is greater than permitted
 30%|███       | 759/2502 [00:08<00:16, 103.71it/s][22:02:42] Explicit valence for atom # 1 N, 4, is greater than permitted
[22:02:42] Explicit valence for atom # 1 N, 4, is greater than permitted
 53%|█████▎    | 1329/2502 [00:14<00:09, 128.28it/s][22:02:49] Explicit valence for atom # 1 N, 4, is greater than permitted
[22:02:49] Explicit valence for atom # 3 N, 4, is greater than permitted
 76%|███████▌  | 1896/2502 [00:20<00:05, 116.83it/s][22:02:55] Explicit valence for atom # 22 N, 4, is greater than permitted
[22:02:55] Explicit valence for atom # 5 N, 4, is greater than permitted
 87%|███████

3048


 80%|████████  | 2451/3048 [00:28<00:05, 100.50it/s][22:03:32] Explicit valence for atom # 14 N, 4, is greater than permitted
[22:03:32] Explicit valence for atom # 5 N, 4, is greater than permitted
100%|██████████| 3048/3048 [00:34<00:00, 87.68it/s] 
/var/folders/k3/l23j0sk144l_frxntwwtfpg40000gn/T/ipykernel_40325/3834039767.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_high_wc['len_linkers'] = len_linkers


4164


 37%|███▋      | 1555/4164 [00:10<00:14, 178.14it/s][22:03:49] Explicit valence for atom # 14 N, 4, is greater than permitted
[22:03:49] Explicit valence for atom # 14 N, 4, is greater than permitted
100%|██████████| 4164/4164 [01:02<00:00, 66.78it/s] 
/var/folders/k3/l23j0sk144l_frxntwwtfpg40000gn/T/ipykernel_40325/3834039767.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_high_wc['len_linkers'] = len_linkers


1911


 14%|█▍        | 275/1911 [00:02<00:22, 71.91it/s] [22:04:43] Explicit valence for atom # 21 N, 4, is greater than permitted
[22:04:43] Explicit valence for atom # 11 N, 4, is greater than permitted
[22:04:43] Explicit valence for atom # 32 N, 4, is greater than permitted
 33%|███▎      | 632/1911 [00:05<00:12, 106.34it/s][22:04:46] Explicit valence for atom # 6 N, 4, is greater than permitted
[22:04:46] Explicit valence for atom # 9 N, 4, is greater than permitted
 76%|███████▌  | 1447/1911 [00:35<01:14,  6.27it/s] [22:05:16] Explicit valence for atom # 17 N, 4, is greater than permitted
[22:05:16] Explicit valence for atom # 1 N, 4, is greater than permitted
100%|██████████| 1911/1911 [00:43<00:00, 43.83it/s]
